## What is MediaPipe ?
![image](https://google.github.io/mediapipe/images/mediapipe_small.png)

MediaPipe offers ready-to-use yet customizable Python solutions as a prebuilt Python package.

Refer - [here](https://google.github.io/mediapipe/)

## Step 1: Install Libraries and Import Dependencies

In [1]:
!pip install mediapipe opencv-python

     |████████████████████████████████| 31.5 MB 231 kB/s eta 0:00:01


In [2]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
import csv
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import pickle

## Step 2: Define Utils

In [3]:
mp_drawing = mp.solutions.drawing_utils # For Drawing the cordinates
mp_hands = mp.solutions.hands # Solution specific for hand coordinate

## Step 3: Make some Detection

In [7]:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(min_detection_confidence=0.5,min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()

        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)



#         image_height, image_width, _ = image.shape

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        cv2.imshow('MediaPipe Hands', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
# cv2.destroyAllWindows()

[ WARN:0] global /tmp/pip-req-build-fvfwe_ss/opencv/modules/videoio/src/cap_v4l.cpp (890) open VIDEOIO(V4L2:/dev/video0): can't open camera by index


## Step 4: Capture Landmarks & Export to CSV

In [5]:
import csv
import os
import numpy as np

In [6]:
num_coords = len(hand_landmarks.landmark)
num_coords

NameError: name 'hand_landmarks' is not defined

In [ ]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val)]

In [ ]:
landmarks

In [ ]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [ ]:
class_name = "Three"

In [ ]:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(min_detection_confidence=0.5,min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
        

        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)



        image_height, image_width, _ = image.shape

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
        try:
            hand = hand_landmarks.landmark
            hand_row = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in hand]).flatten())
            
            row = hand_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
            
   
        cv2.imshow('MediaPipe Hands 4', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

## Step 5: Read in Collected Data and Process

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('coords.csv')

X = df.drop('class', axis=1) # features
y = df['class'] # target value

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

## Step 6: Train Custom Model Using Scikit Learn

In [ ]:

from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [ ]:

pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

## Step 7: Train Machine Learning Classification Model

In [ ]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [ ]:
fit_models['rc'].predict(X_test)

## Step 8: Evaluate and Serialize Model

In [ ]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle

In [ ]:

for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

In [ ]:
fit_models['rf'].predict(X_test)

In [ ]:

with open('Numbers.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

## Step 9: Make Detections with Model

In [ ]:
with open('Numbers.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(min_detection_confidence=0.5,min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
       
        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)



#         image_height, image_width, _ = image.shape

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        try:
            hand = hand_landmarks.landmark
            hand_row = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in hand]).flatten())

            row = hand_row
#             print(row)


            X = pd.DataFrame([row])
            hand_language_class = model.predict(X)[0]
            hand_language_prob = model.predict_proba(X)[0]
            print(hand_language_class, hand_language_prob)



            coords = tuple(np.multiply(
                            np.array(
                                (hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x, 
                                 hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y))
                        , [640,480]).astype(int))

            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(hand_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, hand_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)

            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, hand_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(hand_language_prob[np.argmax(hand_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        except:
            pass
            
   
        cv2.imshow('MediaPipe Hands 2', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()